### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name"])
school_data_complete = school_data_complete.rename(columns={"school_name":"School Name", "type": "School Type", "size": "Total Students","budget": "Total School Budget"}).reset_index(drop=True)

#Ensuring that the data is complete
#student_data.count()

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
#Storing Values to make the data frame more readable
Total_Students=student_data["Student ID"].count()
Students_Passing_Math=student_data.loc[student_data["math_score"]>=70,:]["Student ID"].count()
Total_Math_Score=student_data["math_score"].sum()
Students_Passing_Reading=student_data.loc[student_data["reading_score"]>=70,:]["Student ID"].count()
Total_Reading_Score=student_data["reading_score"].sum()
Students_Passing_Both=student_data.loc[(student_data["reading_score"]>=70) & (student_data["math_score"]>=70),:]["Student ID"].count()

#District Data Frame
District_Summary_df = pd.DataFrame({
    "Total Schools": [school_data["school_name"].count()],
    "Total Students": [Total_Students],
    "Total Budget": [school_data["budget"].sum()],
    "Average Math Score": [Total_Math_Score/Total_Students],
    "Average Reading Score": [Total_Reading_Score/Total_Students],
    "Percent Passing Math": [Students_Passing_Math/Total_Students*100],
    "Percent Passing Reading": [Students_Passing_Reading/Total_Students*100],
    "Percent Passing Overall": [Students_Passing_Both/Total_Students*100]
})

#Cleaning the Data Frame
Clean_District=District_Summary_df
Clean_District["Total Students"]=Clean_District["Total Students"].map("{:,}".format)
Clean_District["Total Budget"]=Clean_District["Total Budget"].map("${:,.2f}".format)
Clean_District["Average Math Score"]=Clean_District["Average Math Score"].map("{:.2f}".format)
Clean_District["Average Reading Score"]=Clean_District["Average Reading Score"].map("{:.2f}".format)
Clean_District["Percent Passing Math"]=Clean_District["Percent Passing Math"].map("{:.3f}%".format)
Clean_District["Percent Passing Reading"]=Clean_District["Percent Passing Reading"].map("{:.3f}%".format)
Clean_District["Percent Passing Overall"]=Clean_District["Percent Passing Overall"].map("{:.3f}%".format)

Clean_District

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,"39,170","$24,649,428.00",78.99,81.88,74.981%,85.805%,65.172%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
#Creating a grouped object for me to retrieve aggregate values from
school_group=school_data_complete.groupby(["School Name"])
math_group=school_data_complete.loc[school_data_complete["math_score"]>=70,:].groupby(["School Name"])
reading_group=school_data_complete.loc[school_data_complete["reading_score"]>=70,:].groupby(["School Name"])
overall_group=school_data_complete.loc[(school_data_complete["reading_score"]>=70) & (school_data_complete["math_score"]>=70),:].groupby(["School Name"])

#Building out the basic part of the DF, School Name, School Type, Total Students, Total Budget and Budget per student
school_summary=school_data.drop(columns=['School ID']).sort_values(["school_name"])
school_summary=school_summary.rename(columns={"school_name":"School Name", "type": "School Type", "size": "Total Students","budget": "Total School Budget"}).reset_index(drop=True)
school_summary["Per Student Budget"]=(school_summary["Total School Budget"]/school_summary["Total Students"]).map("${:,.2f}".format)

#Adding in Average Math and Reading Scores
school_summary=pd.merge(school_summary,school_group["math_score"].sum(),on="School Name")
school_summary=pd.merge(school_summary,school_group["reading_score"].sum(),on="School Name")
school_summary["Average Math Score"]=(school_summary["math_score"]/school_summary["Total Students"]).map("{:.3f}".format)
school_summary["Average Reading Score"]=(school_summary["reading_score"]/school_summary["Total Students"]).map("{:.3f}".format)
school_summary=school_summary.drop(columns=["math_score","reading_score"])

#Adding in % passing for 3 categories
school_summary=pd.merge(school_summary,math_group["Student ID"].count(),on="School Name")
school_summary["Percent Passing Math"]=(school_summary["Student ID"]/school_summary["Total Students"]*100).map("{:.3f}%".format)
school_summary=school_summary.drop(columns=["Student ID"])

school_summary=pd.merge(school_summary,reading_group["Student ID"].count(),on="School Name")
school_summary["Percent Passing Reading"]=(school_summary["Student ID"]/school_summary["Total Students"]*100).map("{:.3f}%".format)
school_summary=school_summary.drop(columns=["Student ID"])

school_summary=pd.merge(school_summary,overall_group["Student ID"].count(),on="School Name")
school_summary["Percent Overall Passing"]=(school_summary["Student ID"]/school_summary["Total Students"]*100).map("{:.3f}%".format)
school_summary=school_summary.drop(columns=["Student ID"])

#Cleaning up the format of the original columns
school_summary["Total Students"]=school_summary["Total Students"].map("{:,}".format)
school_summary["Total School Budget"]=school_summary["Total School Budget"].map("${:,.2f}".format)

school_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048,81.034,66.680%,81.933%,54.642%
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.133%,97.040%,91.335%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.988%,80.739%,53.204%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.103,80.746,68.310%,79.299%,54.290%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.392%,97.139%,90.599%
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.290,80.934,66.753%,80.863%,53.528%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.506%,96.253%,89.227%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.684%,81.316%,53.514%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072,80.966,66.058%,81.222%,53.539%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595%,95.946%,90.541%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
school_summary.sort_values(["Percent Overall Passing"],ascending=False,ignore_index=True).head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
0,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.133%,97.040%,91.335%
1,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418,83.849,93.272%,97.309%,90.948%
2,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.392%,97.139%,90.599%
3,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274,83.989,93.868%,96.540%,90.583%
4,Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595%,95.946%,90.541%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [6]:
school_summary.sort_values(["Percent Overall Passing"],ignore_index=True).head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
0,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.843,80.745,66.367%,80.220%,52.988%
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.988%,80.739%,53.204%
2,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.684%,81.316%,53.514%
3,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.290,80.934,66.753%,80.863%,53.528%
4,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072,80.966,66.058%,81.222%,53.539%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
#All calculated inline from the merged table at the beginning of the notebook
math_df = pd.DataFrame({
    "9th": (school_data_complete.loc[school_data_complete["grade"]=='9th',:].groupby(["School Name"])["math_score"].sum()/school_data_complete.loc[school_data_complete["grade"]=='9th',:].groupby(["School Name"])["math_score"].count()).map("{:.3f}".format),
    "10th": (school_data_complete.loc[school_data_complete["grade"]=='10th',:].groupby(["School Name"])["math_score"].sum()/school_data_complete.loc[school_data_complete["grade"]=='10th',:].groupby(["School Name"])["math_score"].count()).map("{:.3f}".format),
    "11th":(school_data_complete.loc[school_data_complete["grade"]=='11th',:].groupby(["School Name"])["math_score"].sum()/school_data_complete.loc[school_data_complete["grade"]=='11th',:].groupby(["School Name"])["math_score"].count()).map("{:.3f}".format),
    "12th":(school_data_complete.loc[school_data_complete["grade"]=='12th',:].groupby(["School Name"])["math_score"].sum()/school_data_complete.loc[school_data_complete["grade"]=='12th',:].groupby(["School Name"])["math_score"].count()).map("{:.3f}".format)
})
math_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
#All calculated inline from the merged table at the beginning of the notebook
reading_df = pd.DataFrame({
    "9th": (school_data_complete.loc[school_data_complete["grade"]=='9th',:].groupby(["School Name"])["reading_score"].sum()/school_data_complete.loc[school_data_complete["grade"]=='9th',:].groupby(["School Name"])["reading_score"].count()).map("{:.3f}".format),
    "10th": (school_data_complete.loc[school_data_complete["grade"]=='10th',:].groupby(["School Name"])["reading_score"].sum()/school_data_complete.loc[school_data_complete["grade"]=='10th',:].groupby(["School Name"])["reading_score"].count()).map("{:.3f}".format),
    "11th":(school_data_complete.loc[school_data_complete["grade"]=='11th',:].groupby(["School Name"])["reading_score"].sum()/school_data_complete.loc[school_data_complete["grade"]=='11th',:].groupby(["School Name"])["reading_score"].count()).map("{:.3f}".format),
    "12th":(school_data_complete.loc[school_data_complete["grade"]=='12th',:].groupby(["School Name"])["reading_score"].sum()/school_data_complete.loc[school_data_complete["grade"]=='12th',:].groupby(["School Name"])["reading_score"].count()).map("{:.3f}".format)
})
reading_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [26]:
#Duplicate the School Data Complete
School_Spending=school_data_complete
School_Spending["Per Student Budget"]=School_Spending["Total School Budget"]/School_Spending["Total Students"]

#Creating evenly disributed bins
minimum=School_Spending["Per Student Budget"].min()
maximum=School_Spending["Per Student Budget"].max()
diff=(maximum-minimum)/4
bins=[minimum]
for i in range(1,4):
    bins.append(minimum+i*diff)
bins.append(maximum)
Budget_Category=["Low","Medium-Low","Medium-High","High"]


#Add the bins to our data
School_Spending["Budget Category"]=pd.cut(School_Spending["Per Student Budget"],bins,labels=Budget_Category,include_lowest=False)
Spending_Group=School_Spending.groupby(["Budget Category"])

#Data frame to display results
spending_df = pd.DataFrame({
    "Average Math Score": Spending_Group["math_score"].mean().map("{:.3f}".format),
    "Average Reading Score": Spending_Group["reading_score"].mean().map("{:.3f}".format),
    "Percent Passing Math":(School_Spending.loc[School_Spending["math_score"]>=70,:].groupby(["Budget Category"])["Student ID"].count()/Spending_Group["Student ID"].count()*100).map("{:.3f}%".format),
    "Percent Passing Reading":(School_Spending.loc[School_Spending["reading_score"]>=70,:].groupby(["Budget Category"])["Student ID"].count()/Spending_Group["Student ID"].count()*100).map("{:.3f}%".format),
    "Percent Passing Overall":(School_Spending.loc[(School_Spending["reading_score"]>=70) & (School_Spending["math_score"]>=70),:].groupby(["Budget Category"])["Student ID"].count()/Spending_Group["Student ID"].count()*100).map("{:.3f}%".format)
})


spending_df

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
Budget Category,,,,,
Low,83.413,83.950,93.611%,96.769%,90.673%
Medium-Low,83.529,83.838,94.124%,95.887%,90.121%
Medium-High,78.484,81.668,72.765%,85.397%,62.834%
High,77.419,81.147,68.335%,81.824%,56.074%


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
